In [ ]:
import pandas as pd
import os
from datetime import datetime
import mydashtools as dt
import plotly.express as px
import re
pd.options.plotting.backend = "plotly"
if dt.is_notebook():
    __file__ = "notebook/"

zelty = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../zelty/zelty.csv", sep=";",decimal=',',low_memory=False, index_col=False)

limit_year = None
if os.getenv("LIMIT_YEAR") is not None:
    limit_year = os.getenv("LIMIT_YEAR")
if limit_year:
    zelty = zelty[zelty['date'] <= limit_year+'-12-31']

zelty['category'] = zelty['category'].fillna('Catégorie inconnue')
zelty['date'] = pd.to_datetime(zelty['date'])
zelty.rename(columns={'category': 'Type de produit', 'price': 'Prix'}, inplace=True)
if dt.is_notebook():
    print(zelty.head())

In [ ]:
chiffredaffaire = zelty.groupby(['date', 'Type de produit'])[['Prix']].sum().unstack('Type de produit')

In [ ]:
chiffredaffaire_par_an = chiffredaffaire.resample('YS').sum().fillna(0)
chiffredaffaire_par_an['Année'] = chiffredaffaire_par_an.index.year
chiffredaffaire_par_an.reset_index(inplace=True)
chiffredaffaire_par_an.set_index('Année', inplace=True)
# = chiffredaffaire.columns.droplevel(0)
chiffredaffaire_par_an.columns = chiffredaffaire_par_an.columns.droplevel(0)
del chiffredaffaire_par_an['']

fig = px.bar(chiffredaffaire_par_an)
fig.update_layout(legend_title="Type de produit",xaxis_title=None,yaxis_title=None,separators=", ")
fig.update_yaxes(tickformat=",d",ticksuffix=" €")
dt.fig_save_or_show(fig, 'recettes_01_ca_par_categories_et_an', limit_year)

In [ ]:
pid2produit = zelty.groupby(['pid'])['produit'].last().to_dict()

In [ ]:
chiffredaffaire_12_mois = zelty[zelty.date > str(max(zelty.date).year - 1)]
chiffredaffaire_12_mois = chiffredaffaire_12_mois.groupby(['date', 'pid'])[['Prix']].sum().reset_index()
chiffredaffaire_12_mois['produit'] = chiffredaffaire_12_mois['pid'].map(pid2produit)
chiffredaffaire_12_mois['produit'] = chiffredaffaire_12_mois['produit'].apply(lambda x: re.sub(r'  *', ' ', x)[:25].title())
chiffredaffaire_12_mois = chiffredaffaire_12_mois.reset_index()
chiffredaffaire_12_mois = chiffredaffaire_12_mois.groupby(['date', 'produit'])[['Prix']].sum()
chiffredaffaire_12_mois = chiffredaffaire_12_mois.unstack('produit')
chiffredaffaire_12_mois = chiffredaffaire_12_mois.resample('MS').sum()
chiffredaffaire_12_mois.columns = chiffredaffaire_12_mois.columns.droplevel(0)

fig = px.bar(chiffredaffaire_12_mois.dropna())
fig.update_layout(legend_title="Produit",xaxis_title=None,yaxis_title=None,separators=', ')
fig.update_yaxes(tickformat=",d",ticksuffix=" €")
dt.fig_save_or_show(fig, 'recettes_02_ca_par_produits_12_mois', limit_year)

In [ ]:
chiffredaffaire_total_by_year = chiffredaffaire.stack('Type de produit').reset_index().groupby('date')[['Prix']].sum().resample('Y').sum()
chiffredaffaire_total_by_year.index = chiffredaffaire_total_by_year.index.year

chiffredaffaire_a_date = chiffredaffaire.stack('Type de produit').reset_index().groupby('date')[['Prix']].sum()
chiffredaffaire_a_date['a_date'] = chiffredaffaire_a_date.index.month*100+chiffredaffaire_a_date.index.day
chiffredaffaire_a_date = chiffredaffaire_a_date[chiffredaffaire_a_date['a_date'] < max(zelty.date).month*100+max(zelty.date).day]
del chiffredaffaire_a_date['a_date']
chiffredaffaire_a_date = chiffredaffaire_a_date.resample('Y').sum()
chiffredaffaire_a_date.index = chiffredaffaire_a_date.index.year
chiffredaffaire_a_date
libelle_a_date = "à date du %02d/%02d" % (max(zelty.date).day, max(zelty.date).month)
chiffredaffaire_total_by_year[libelle_a_date] = chiffredaffaire_a_date['Prix']
chiffredaffaire_total_by_year.fillna(0, inplace=True)
chiffredaffaire_total_by_year['Toute l\'année'] = chiffredaffaire_total_by_year['Prix']
del chiffredaffaire_total_by_year['Prix']
del chiffredaffaire_total_by_year['Toute l\'année'][max(chiffredaffaire_total_by_year.index)]


import plotly.graph_objects as go
fig = go.Figure(data=[
    go.Bar(name=libelle_a_date, x=chiffredaffaire_total_by_year.index, y=chiffredaffaire_total_by_year[libelle_a_date]),
    go.Bar(name='Toute l\'année', x=chiffredaffaire_total_by_year.index, y=chiffredaffaire_total_by_year['Toute l\'année'])
])
fig.update_layout(xaxis_title=None,yaxis_title=None,separators=', ')
fig.update_yaxes(tickformat=",d",ticksuffix=" €")
dt.fig_save_or_show(fig, 'recettes_03_ca_total_par_an', limit_year)

In [ ]:
limit_date = max(zelty.date).replace(year=max(zelty.date).year - 1)
chiffredaffaire_year = chiffredaffaire[chiffredaffaire.index.to_period("M") >= str(limit_date.year)+'-'+str(limit_date.month)]
chiffredaffaire_year.columns = chiffredaffaire_year.columns.droplevel(0)
chiffredaffaire_year = chiffredaffaire_year.groupby('date').sum()
chiffredaffaire_year['total'] = pd.DataFrame(chiffredaffaire_year.stack('Type de produit'), columns=['total']).groupby('date').sum()

In [ ]:
events = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../events/barco_events_all.csv", sep=";",decimal=',',low_memory=False, index_col=False)
events['date'] = pd.to_datetime(events['date'])
events.set_index('date', inplace=True)
events_by_day = events.resample('D')['titre'].apply(' / '.join).reset_index()
events_by_day['date'] = events_by_day['date'].apply(lambda x: str(x)[0:10])
events_by_day.set_index('date', inplace=True)
d2e = events_by_day[events_by_day['titre'] != '']['titre'].to_dict()

In [ ]:
chiffredaffaire_year['évènement'] = chiffredaffaire_year.index.strftime('%Y-%m-%d').map(d2e).fillna('')
chiffredaffaire_year['jour'] = chiffredaffaire_year.index.strftime('%A')

fig = px.bar(chiffredaffaire_year, y=['total'], hover_data=['évènement', 'jour'])
fig.update_layout(xaxis_title=None,yaxis_title=None,separators=', ')
fig.update_yaxes(tickformat=",d",ticksuffix=" €")
dt.fig_save_or_show(fig, 'recettes_04_ca_quotidien_12_mois', limit_year)

In [ ]:
from datetime import datetime 
from dateutil.relativedelta import relativedelta 

chiffredaffaire_top_year = chiffredaffaire_year.sort_values(by='total', ascending=False)
chiffredaffaire_top_year = chiffredaffaire_top_year[['total']].head(50)

chiffredaffaire_top_year['mois_courant'] = chiffredaffaire_top_year.index >= max(chiffredaffaire_top_year.index) + relativedelta(months=-1)
chiffredaffaire_top_year = chiffredaffaire_top_year.reset_index().set_index(['date', 'mois_courant'])
chiffredaffaire_top_year = chiffredaffaire_top_year.unstack('mois_courant')
chiffredaffaire_top_year.columns = chiffredaffaire_top_year.columns.droplevel(0)
chiffredaffaire_top_year['total'] = chiffredaffaire_top_year[True].fillna(0) + chiffredaffaire_top_year[False].fillna(0)
chiffredaffaire_top_year = chiffredaffaire_top_year.sort_values(by='total', ascending=False)
chiffredaffaire_top_year['évènement'] = chiffredaffaire_top_year.index.strftime('%Y-%m-%d').map(d2e).fillna('<aucun>')
chiffredaffaire_top_year['jour'] = chiffredaffaire_top_year.index.strftime('%A')
chiffredaffaire_top_year.index = chiffredaffaire_top_year.index.strftime('%d/%m/%Y')
del chiffredaffaire_top_year['total']

fig = px.bar(chiffredaffaire_top_year, y=[False,True], hover_data=['évènement', 'jour'])
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None,separators=', ')
fig.update_yaxes(tickformat=",d",ticksuffix=" €")
dt.fig_save_or_show(fig, 'recettes_05_ca_quotidien_top50_12_mois', limit_year)

In [ ]:
marges = {2017:0.46,
          2018:0.4795,
          2019:0.5012,
          2020:0.3679,
          2021:0.5078,
          2022:0.3363,
          2023:0.4922,
          2024:0.4922
         }


In [ ]:
charges = {2017:50353,
           2018:50353,
           2019:50786,
           2020:45973-33171,
           2021:30298-40000,
           2022:45886,
           2023:46195,
           2024:46195
          }


In [ ]:
benefices = chiffredaffaire.stack('Type de produit')[['Prix']].groupby('date').sum().resample('MS').sum()
benefices['Année'] = benefices.index.year
benefices['marge'] = benefices['Année'].map(marges)
benefices['charges_fixes'] = benefices['Année'].map(charges)/12
benefices = benefices.groupby(['date','Année']).sum()
benefices['taux_ht'] = 0.8333
benefices['total'] = benefices['Prix'] * benefices['marge'] * benefices['taux_ht'] - benefices['charges_fixes']
del benefices['Prix']
del benefices['marge']
del benefices['charges_fixes']
del benefices['taux_ht']
benefices = benefices.unstack('Année').fillna(0)
benefices.columns = benefices.columns.droplevel(0)

fig = px.bar(benefices)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None,separators=', ')
fig.update_yaxes(tickformat=",d",ticksuffix=" €")
dt.fig_save_or_show(fig, 'recettes_06_benefice', limit_year)

In [ ]:
if dt.is_notebook():
    print(benefices.sum())